In [1]:
#Libraries.
import pandas as pd

### Data Import.

In [7]:
#read the data. titanic data, from Kaggle.
data_train = pd.read_csv('./data/titanic_train.csv')
data_test  = pd.read_csv('./data/titanic_test.csv')

data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Preprocessing.

In [40]:
data_train_cpy = data_train.copy()  #should copy before manipulate!!

#1. Numerical data.
#1) Null check.
data_train_cpy.info()  #Null values in 'Age', 'Cabin', 'Embarked'.
#1 - Delete row without 'Embarked', cuz only 2 samples omit this col.
data_train_cpy = data_train_cpy.dropna(subset=['Embarked'])
#2 - Delete col 'Cabin'.
data_train_cpy = data_train_cpy.drop(columns=['Cabin'])
#3 - Fill nulls in 'Age' with mean value.
mean_age              = data_train_cpy['Age'].mean()
data_train_cpy['Age'] = data_train_cpy['Age'].fillna(mean_age)
data_train_cpy.info()

#Null check with sklearn.impute.
cols_cat           = data_train_cpy.select_dtypes('object').columns  #col names of cat cols.
data_train_cpy_num = data_train_cpy.drop(columns=cols_cat)           #get num cols.
cols_num           = data_train_cpy_num.columns                      #col names of num cols.
data_train_cpy_cat = data_train_cpy.drop(columns=cols_num)           #get cat cols.

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')  #fill null values with median value for each col.
imputer.fit(data_train_cpy_num)             #automatically calculates each median.
imputer.statistics_                         #save median values for each num col.
data_train_prep_num = imputer.transform(data_train_cpy_num)   #actually fill in given data.
#data_train_prep = imputer.fit_transform(data_train_cpy_num)   #at once!
data_train_prep_num = pd.DataFrame(data_train_prep, columns=cols_num, index=data_train_cpy_num.index)

#2) Feature scaling.
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mmx_sclr = MinMaxScaler()
mmx_sclr.fit_transform(data_train_cpy_num)

#2. Categorical data.
#1) Null check.
#2) Cat -> Num.
    #Ordinal encoder.
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
#data_train_prep_cat = ord_enc.fit_transform(data_train_cpy_cat)
#ord_enc.categories_    #saves which ordinal number is assigned for each col's value.
    #One-hot encoder.
from sklearn.preprocessing import OneHotEncoder
one_enc = OneHotEncoder()
data_train_prep_cat = one_enc.fit_transform(data_train_cpy_cat)    
one_enc.categories_
data_train_prep_cat.toarray()   #scipy sparse matrix -> np.ndarray

#3. Measurement unit check.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])